In [4]:
import requests

import base64
from pyDes import des, ECB, PAD_PKCS5
import hashlib
import xml.etree.ElementTree as ET

import time
import urllib.parse
from datetime import datetime, timedelta

In [5]:
def DESEnCode(mystr, deskey):
    mytext = mystr.encode('utf-8')
    key = deskey.encode('utf-8')
    iv = deskey.encode('utf-8')
    
    # DES Encryption
    k = des(key, ECB, IV=iv, pad=None, padmode=PAD_PKCS5)
    cryptoText = k.encrypt(mytext)
    
    # Encoding to base64
    encoded = base64.b64encode(cryptoText).decode('utf-8')
    return encoded

def BuildMD5(mystr):
    h = hashlib.md5()
    h.update(mystr.encode('utf-8'))
    data = h.hexdigest()
    return data

In [6]:
defaultSPSecretKey = "494DA5A7FC954AA4973456A08609C2F9"
defaultSPMd5key    = "GgaIMaiNNtg"
defaultSPEncrypKey = "g9G16nTs"

# Get the current date
today = datetime.now()

# Calculate the date one year ago from today
yesterday = today - timedelta(days=1)

# Format the dates as strings in the format "YYYYMMDDHHMMSS"
dateFrom = yesterday.strftime("%Y%m%d%H%M%S")
dateTo = today.strftime("%Y%m%d%H%M%S")

In [7]:
timeStr = time.strftime("%Y%m%d%H%M%S")
qs = "method=GetAllBetDetailsForTimeInterval"
qs = qs + "&Key=" + '494DA5A7FC954AA4973456A08609C2F9'
qs = qs + "&Time=" + timeStr
qs = qs + "&FromTime=" + dateFrom
qs = qs + "&ToTime=" + dateTo

In [8]:
q = DESEnCode(qs, defaultSPEncrypKey)
sig = BuildMD5(qs + defaultSPMd5key + timeStr + defaultSPSecretKey)

data = urllib.parse.urlencode({'q': q, 's': sig})

In [9]:
url = 'http://localhost:8800/'

res = requests.post(url, data=data)

In [10]:
# Parse the XML response
root = ET.fromstring(res.text)

In [12]:
from pyspark.sql import SparkSession
from datetime import date
from pyspark.sql import Row
from pyspark.sql.functions import lit

In [13]:
spark = SparkSession.builder \
    .appName("HadoopSparkTest") \
    .master("local") \
    .getOrCreate()

23/08/09 14:12:56 WARN Utils: Your hostname, tns-des180 resolves to a loopback address: 127.0.1.1; using 172.24.31.35 instead (on interface enp2s0)
23/08/09 14:12:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/09 14:13:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
df = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rowTag", "HistoryResponse") \
    .xml(res.text)

AttributeError: 'DataFrameReader' object has no attribute 'xml'

In [19]:
df = spark.read
  .option("rowTag", "HistoryResponse")
  .schema(schema)
  .xml(res.text)

df.show()

SyntaxError: invalid syntax (2191294854.py, line 1)

In [11]:
# Access elements in the XML and extract data
error_msg_id = root.find('ErrorMsgId').text
error_msg = root.find('ErrorMsg').text

In [19]:
# Access nested elements
bet_details = []
for b in root.findall('.//BetDetail'):
    # Create a dictionary to store the data for each bet detail
    bet_data = {
        'BetTime': b.find('BetTime').text,
        'PayoutTime': b.find('PayoutTime').text,
        'BetTime': b.find('BetTime').text,
        'PayoutTime': b.find('PayoutTime').text,
        'Username': b.find('Username').text,
        'HostID': b.find('HostID').text,
        'Detail': b.find('Detail').text,
        'GameID': b.find('GameID').text,
        'Round': b.find('Round').text,
        'Set': b.find('Set').text,
        'BetID': b.find('BetID').text,
        'BetAmount': b.find('BetAmount').text,
        'Rolling': b.find('Rolling').text,
        'ResultAmount': b.find('ResultAmount').text,
        'Balance': b.find('Balance').text,
        'GameType': b.find('GameType').text,
        'BetType': b.find('BetType').text,
        'BetSource': b.find('BetSource').text,
        'TransactionID': b.find('TransactionID').text,
        'GameResult': b.find('GameResult').text,
        'State': b.find('State').text,
    }
    bet_details.append(bet_data)

# Create a dictionary to store the entire response
history = {
    'ErrorMsgID': error_msg_id,
    'ErrorMsg': error_msg,
    'BetDetailList': bet_details,
}

# You can then work with the data in the 'history_response' dictionary
print(history)

{'ErrorMsgID': '0', 'ErrorMsg': 'Success', 'BetDetailList': [{'BetTime': '2023-08-01 12:00:00', 'PayoutTime': '2023-08-01 13:00:00', 'Username': 'user1', 'HostID': '1', 'Detail': 'Bet on slot machine', 'GameID': 'slot123', 'Round': '1', 'Set': '1', 'BetID': '123456789', 'BetAmount': '100', 'Rolling': '50', 'ResultAmount': '150', 'Balance': '500', 'GameType': 'slot', 'BetType': '1', 'BetSource': '1', 'TransactionID': '987654321', 'GameResult': 'Win', 'State': 'true'}, {'BetTime': '2023-08-01 12:00:00', 'PayoutTime': '2023-08-01 13:00:00', 'Username': 'user2', 'HostID': '2', 'Detail': 'Bet on slot machine', 'GameID': 'fishing123', 'Round': '1', 'Set': '1', 'BetID': '123456789', 'BetAmount': '100', 'Rolling': '50', 'ResultAmount': '150', 'Balance': '500', 'GameType': 'fishing', 'BetType': '1', 'BetSource': '1', 'TransactionID': '987654321', 'GameResult': 'Win', 'State': 'true'}]}


# Creating a Table

In [41]:
temp = {
    "database_name" : "temp_collector"
}

In [43]:
# Connection parameters
hive_params = {
    "host" : "172.17.0.1",
    "port" : 10000,  # Default Hive server port
    "database" : temp['database_name'],
    "username" : "scott", 
    "password" : "tiger",
    "auth" : 'CUSTOM'
}

In [44]:

# Establish the connection
conn = hive.Connection(**hive_params)


In [52]:
cursor = conn.cursor()

In [58]:
# Create a table for simpleplay_wager
hive_sql = """CREATE TABLE IF NOT EXISTS simpleplay_wager (
    bet_time STRING,
    payout_time STRING,
    username STRING,
    host_id SMALLINT,
    detail STRING,
    game_id STRING,
    round_num INT,
    set_num INT,
    bet_id BIGINT,
    bet_amount DOUBLE,
    rolling DOUBLE,
    result_amount DOUBLE,
    balance DOUBLE,
    game_type STRING,
    bet_type INT,
    bet_source INT,
    transaction_id BIGINT,
    game_result STRING,
    state BOOLEAN
    )"""

In [57]:
# Execute SQL
cursor.execute(hive_sql)

# Saving the file to the table

In [68]:
insert_query_template = """
INSERT INTO simpleplay_wager (
    bet_time,
    payout_time,
    username,
    host_id,
    detail,
    game_id,
    round_num,
    set_num,
    bet_id,
    bet_amount,
    rolling,
    result_amount,
    balance,
    game_type,
    bet_type,
    bet_source,
    transaction_id,
    game_result,
    state
)
VALUES (
    %(BetTime)s,
    %(PayoutTime)s,
    %(Username)s,
    %(HostID)s,
    %(Detail)s,
    %(GameID)s,
    %(Round)s,
    %(Set)s,
    %(BetID)s,
    %(BetAmount)s,
    %(Rolling)s,
    %(ResultAmount)s,
    %(Balance)s,
    %(GameType)s,
    %(BetType)s,
    %(BetSource)s,
    %(TransactionID)s,
    %(GameResult)s,
    %(State)s
)
"""

In [69]:
for row_data in bet_details:
    cursor.execute(insert_query_template, row_data)

In [ ]:
conn.commit()
conn.close()